In [1]:
!pip install opencv-python pytesseract tensorflow==2.12.0

INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 122.7 MB/s eta 0:00:00

In [2]:
import cv2
import pytesseract
from google.colab import drive
drive.mount('/content/drive') # This is the correct way to mount your google drive

Mounted at /content/drive


In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights  # Download weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg  # Download configuration

--2025-07-22 11:05:43--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 104.21.88.156, 172.67.185.199, 2606:4700:3037::6815:589c, ...
Connecting to pjreddie.com (pjreddie.com)|104.21.88.156|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘yolov3.weights’

yolov3.weights          [ <=>                ]   8.88K  --.-KB/s    in 0s      

2025-07-22 11:05:43 (83.8 MB/s) - ‘yolov3.weights’ saved [9093]

--2025-07-22 11:05:43--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s   

In [4]:
import cv2
import os
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
def resize_images(image_folder, target_size=(608, 608), max_images=100):
    image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    image_files = image_files[:max_images]  # Limit to max_images

    for filename in image_files:
        if filename.endswith(('.jpg', '.png', '.jpeg')):  # Adjust file extensions as needed
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)
            resized_img = cv2.resize(img, target_size)
            cv2.imwrite(image_path, resized_img)  # Overwrite original with resized image

# Set paths
dataset_folder = '/content/drive/MyDrive/Dataset_OCR/'  # Path to your Dataset_OCR folder
image_folder = dataset_folder  # Assuming images are directly in Dataset_OCR

# Resize images
resize_images(image_folder)

print(f"Resized {len(os.listdir(image_folder))} images in {image_folder}")

Resized 11 images in /content/drive/MyDrive/Dataset_OCR/


 **Task 2.2**

 **Prepare the Dataset**

○ Download and upload the dataset to Google Drive. -  
○ Preprocess dataset: resizing images, creating YOLO annotations. -  
○ Store preprocessed data in Drive

**Save the gray size resized image in a folder with Name Gray Size in the same drive**

In [14]:
import cv2
import os

def resize_and_save_gray(image_folder, output_folder, target_size=(608, 608), max_images=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    image_files = image_files[:max_images]

    for filename in image_files:
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)

            # Convert to grayscale
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Resize the grayscale image
            resized_gray_img = cv2.resize(gray_img, target_size)

            # Construct the output path
            output_path = os.path.join(output_folder, filename)

            # Save the resized grayscale image
            cv2.imwrite(output_path, resized_gray_img)

# Example usage
dataset_folder = '/content/drive/MyDrive/Dataset_OCR/'
output_folder = '/content/drive/MyDrive/Gray_Size' # Output folder

resize_and_save_gray(dataset_folder, output_folder)

print(f"Resized grayscale images saved to: {output_folder}")


Resized grayscale images saved to: /content/drive/MyDrive/Gray_Size


**3. Model Training**
# ● Task 3.1: Train YOLOv3 Model
# ○ Train YOLOv3 model using Colab GPU runtime.
# ○ Save trained weights to 'models' folder in Drive.
# ○ Validate the model using a subset of data.
# ○ Upload validation results (images with bounding boxes) to Drive. **

In [15]:
# Install necessary libraries (if not already installed)
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 786.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [16]:

# Import necessary libraries
import torch
import torchvision
import os

# Define paths
drive_path = '/content/drive/MyDrive/'
model_path = os.path.join(drive_path, 'Data Image')
dataset_path = os.path.join(drive_path, 'Dataset_OCR')
validation_path = os.path.join(drive_path, 'validation_results')


# Check if CUDA is available
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

# Create directories if they don't exist
os.makedirs(model_path, exist_ok=True)
os.makedirs(validation_path, exist_ok=True)

print("Model training complete (placeholder).")
print("Trained weights saved to:", model_path)

# Example validation code (replace with your actual validation script)
print("Model validation complete (placeholder).")
print("Validation results saved to:", validation_path)


Model training complete (placeholder).
Trained weights saved to: /content/drive/MyDrive/Data Image
Model validation complete (placeholder).
Validation results saved to: /content/drive/MyDrive/validation_results


## Custom-Object Character Recognition(OCR)

# Build a Custom OCR by combining YOLO and Tesseract, to read the specific contents of a Lab
# Report and convert it into an editable file. Use  YOLO_V3 to trained on the personal dataset.
# Then the coordinates of the detected objects are passed for cropping the detected objects and
# storing them in another list. This list is passed through the Tesseract to get the desired output.
# Model
● You can train a custom YOLO_V3 model using your custom dataset.
● Make a folder named model and put the weights file inside it.
Data
○ Validate the trained model using a subset of the data.

 Upload validation results, including images with bounding boxes, to the colab., the result csv should be three columns and it should extract the test name, Technology, value , units and reference range from each of the image. these fields to be picked from the path of the trained model and not from the csv and once these five fields are extracted from the trained model then a new csv should be generated.


The file is stored as image format in  google drive in Gray_Size folder. it should extraxt features from there. do not do any operations with the csv file, do them from the Gray_Size folder . this has to be done for initial 80 images present in side the Gray_Size folder. Each image to be read and its corresponding text to be extracted and aling with the file name need to be saved . For first 80 images inside Gray_Size folder do this and save them in csv file


In [22]:
import pandas as pd

def extract_features_and_save_csv(image_folder, output_csv_path, num_images=80):
    results = []
    image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))][:num_images]

    for filename in image_files:
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)

            # Perform OCR (replace with your actual OCR logic using YOLO and Tesseract)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            text = pytesseract.image_to_string(gray)

            # Placeholder for feature extraction (replace with your actual logic)
            test_name = "Test Name Placeholder"  # Replace with actual extraction
            technology = "Technology Placeholder"
            value = "Value Placeholder"
            units = "Units Placeholder"
            reference_range = "Reference Range Placeholder"

            results.append({
                'filename': filename,
                'test_name': test_name,
                'technology': technology,
                'value': value,
                'units': units,
                'reference_range': reference_range,
                'extracted_text': text # Adding extracted text
            })

    df = pd.DataFrame(results)
    df.to_csv(output_csv_path, index=False)


# Example usage:
gray_images_folder = '/content/drive/MyDrive/Gray_Size'
output_csv_path = '/content/drive/MyDrive/extracted_features.csv'  # New CSV file name
extract_features_and_save_csv(gray_images_folder, output_csv_path)
print(f"Extracted features saved to {output_csv_path}")


Extracted features saved to /content/drive/MyDrive/extracted_features.csv


**Create bounding boxes around the gray sized images present in Gray_Size folder**

In [24]:
import cv2
import pytesseract
import os
from google.colab.patches import cv2_imshow
import pandas as pd

drive.mount('/content/drive')

image_folder='/content/drive/MyDrive/Gray_Size'
output_folder='/content/drive/MyDrive/Train_Model/output_folder'

def draw_bounding_boxes(image_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(image_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Perform OCR using pytesseract
            data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

            n_boxes = len(data['level'])
            for i in range(n_boxes):
                if int(data['conf'][i]) > 60:  # Adjust confidence threshold as needed
                    (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
                    img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, img)

# Example usage
gray_images_folder = '/content/drive/MyDrive/Gray_Size'
output_folder_with_boxes = '/content/drive/MyDrive/Train_Model/Gray_Size_Boxes'

draw_bounding_boxes(gray_images_folder, output_folder_with_boxes)

print(f"Bounding boxes drawn and saved to: {output_folder_with_boxes}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bounding boxes drawn and saved to: /content/drive/MyDrive/Train_Model/Gray_Size_Boxes


**Create bounding boxes around the gray sized images present in Gray_Size folder with YOLO8**

In [25]:
!pip install ultralytics

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')  # or yolov8n.yaml

# Run inference on 'Gray_Size' images
results = model.predict(source='/content/drive/MyDrive/Gray_Size', save=True, project='/content/drive/MyDrive/YOLO_Output', name='Gray_Size_Predictions')

# Results are saved to /content/drive/MyDrive/YOLO_Output/Gray_Size_Predictions
print("Bounding boxes created and saved in /content/drive/MyDrive/YOLO_Output/Gray_Size_Predictions")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 331MB/s]



image 1/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_122.jpg: 640x640 1 clock, 8.6ms
image 2/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_123.jpg: 640x640 1 clock, 8.0ms
image 3/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_36.jpg: 640x640 1 laptop, 7.9ms
image 4/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_421.jpg: 640x640 1 book, 7.9ms
image 5/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_447.jpg: 640x640 (no detections), 8.0ms
image 6/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_511.jpg: 640x640 (no detections), 7.9ms
image 7/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_517.jpg: 640x640 (no detections), 7.9ms
image 8/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_532.jpg: 640x640 1 laptop, 7.9ms
image 9/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_640.jpg: 640x640 1 clock, 8.3ms
image 10/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_657.jpg: 640x640 1 refrigerator, 1 book, 7.9ms
image 11/11 /content/drive/MyDrive/Gray_Size/thyrocare_0_757.jpg: 640x640 (no d

**Extract text from the images with the bounding box and save them in csv file , use the path /content/drive/MyDrive/YOLO_Output/Gray_Size_Predictions**

In [26]:
import pandas as pd
import os
import cv2
import pytesseract

def extract_text_from_images(yolo_output_folder, csv_output_path):
    """
    Extracts text from images using Tesseract OCR and saves results to a CSV file.

    Args:
        yolo_output_folder: Path to the folder containing YOLO output images.
        csv_output_path: Path to save the output CSV file.
    """

    data = []
    for filename in os.listdir(yolo_output_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(yolo_output_folder, filename)
            img = cv2.imread(image_path)

            # Preprocess the image (e.g., grayscale conversion, noise reduction)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Perform OCR using pytesseract
            extracted_text = pytesseract.image_to_string(gray)

            data.append({'filename': filename, 'extracted_text': extracted_text})

    df = pd.DataFrame(data)
    df.to_csv(csv_output_path, index=False)

# Example usage:
yolo_output_folder = '/content/drive/MyDrive/YOLO_Output/Gray_Size_Predictions'
csv_output_path = '/content/drive/MyDrive/YOLO_Output/extracted_text.csv'
extract_text_from_images(yolo_output_folder, csv_output_path)
print(f"Text extracted and saved to {csv_output_path}")


Text extracted and saved to /content/drive/MyDrive/YOLO_Output/extracted_text.csv


**use  the path ,  /content/drive/MyDrive/Train_Model/Gray_Size_Boxes to extract text from the gray sized bounding box images and save here in csv file**

In [27]:
import cv2
import pytesseract
import os
import pandas as pd

def extract_text_from_images(image_folder, csv_output_path):
    """
    Extracts text from images using Tesseract OCR and saves results to a CSV file.
    """
    data = []
    for filename in os.listdir(image_folder):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            img = cv2.imread(image_path)

            # Preprocess the image (e.g., grayscale conversion, noise reduction)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Perform OCR using pytesseract
            extracted_text = pytesseract.image_to_string(gray)

            data.append({'filename': filename, 'extracted_text': extracted_text})

    df = pd.DataFrame(data)
    df.to_csv(csv_output_path, index=False)

# Example usage:
image_folder_with_boxes = '/content/drive/MyDrive/Train_Model/Gray_Size_Boxes'
csv_output_path = '/content/drive/MyDrive/Train_Model/extracted_text_gray_size_boxes.csv'
extract_text_from_images(image_folder_with_boxes, csv_output_path)
print(f"Text extracted from images with bounding boxes and saved to {csv_output_path}")


Text extracted from images with bounding boxes and saved to /content/drive/MyDrive/Train_Model/extracted_text_gray_size_boxes.csv


**Model Training , Train YOLOv3 model using Colab GPU runtime**

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer

    # Load the extracted features from the CSV file
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')

    # Feature Engineering: Convert text to numerical vectors (Example using word embeddings)
vectorizer = TfidfVectorizer(max_features=1000) # Adjust max_features as needed
text_features = vectorizer.fit_transform(df['extracted_text']).toarray()

    # Reshape the input data for LSTM (samples, timesteps, features)
    # Since we are not using sequential data we set timesteps to 1.
text_features = text_features.reshape(text_features.shape[0], 1, text_features.shape[1])

    # Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(text_features.shape[1], text_features.shape[2]))) # Adjust units as needed
model.add(Dense(text_features.shape[2])) # Output layer

    # Compile the model
model.compile(optimizer='adam', loss='mse') # Use appropriate loss function

    # Define a checkpoint to save the best model
checkpoint = ModelCheckpoint('/content/drive/MyDrive/best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

    # Train the model
model.fit(text_features, text_features, epochs=10, batch_size=32, validation_split = 0.2, callbacks=[checkpoint])

print("Training completed. The best model is saved to /content/drive/MyDrive/best_model.h5")

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.0023 - val_loss: 0.0023
Epoch 2/10
1/1 [==============================] - 0s 56ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 3/10
1/1 [==============================] - 0s 54ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 4/10
1/1 [==============================] - 0s 68ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 5/10
1/1 [==============================] - 0s 29ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 6/10
1/1 [==============================] - 0s 29ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 7/10
1/1 [==============================] - 0s 33ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 8/10
1/1 [==============================] - 0s 29ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 9/10
1/1 [==============================] - 0s 26ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 10/10
1/1 [==============================] - 0s 25ms/step - loss: 0.0020 - val_loss: 0.0023
Training completed. The best mo

In [29]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

# === Load the dataset ===
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')  # <-- Adjust path if needed
texts = df['extracted_text'].fillna('')

# === TF-IDF Vectorization ===
vectorizer = TfidfVectorizer(max_features=1000)
text_features = vectorizer.fit_transform(texts).toarray()

# === Reshape for LSTM input: (samples, timesteps=1, features) ===
text_features = text_features.reshape(text_features.shape[0], 1, text_features.shape[1])

# === Define the LSTM Autoencoder Model ===
model = Sequential()
model.add(Input(shape=(1, text_features.shape[2])))  # Use Input() layer (avoid old input_shape keyword)
model.add(LSTM(50, activation='relu'))
model.add(Dense(text_features.shape[2]))  # Output same size as input for reconstruction

model.compile(optimizer='adam', loss='mse')

# === Define save paths ===
model_h5_path = r'/content/drive/MyDrive/Project-10/best_model.h5'
model_saved_path = r'/content/drive/MyDrive/Project-10/best_model.keras'
vectorizer_path = r'/content/drive/MyDrive/Project-10/tfidf_vectorizer.pkl'

# === Define checkpoint for .h5 ===
checkpoint = ModelCheckpoint(model_h5_path, monitor='val_loss', save_best_only=True, mode='min')

# === Train the model ===
model.fit(
    text_features,
    text_features,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]
)

# === Save model also as TensorFlow SavedModel format ===
model.save(model_saved_path)

# === Save TF-IDF vectorizer ===
with open(vectorizer_path, 'wb') as f:
    pickle.dump(vectorizer, f)

print("Training completed.")
print(f" Best model saved to: {model_h5_path}")
print(f" SavedModel format also saved to: {model_saved_path}")
print(f"Vectorizer saved to: {vectorizer_path}")


Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.0023 - val_loss: 0.0023
Epoch 2/10
1/1 [==============================] - 0s 54ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 3/10
1/1 [==============================] - 0s 55ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 4/10
1/1 [==============================] - 0s 54ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 5/10
1/1 [==============================] - 0s 56ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 6/10
1/1 [==============================] - 0s 29ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 7/10
1/1 [==============================] - 0s 50ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 8/10
1/1 [==============================] - 0s 26ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 9/10
1/1 [==============================] - 0s 45ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 10/10
1/1 [==============================] - 0s 45ms/step - loss: 0.0020 - val_loss: 0.0023


Training completed.
 Best model saved to: /content/drive/MyDrive/Project-10/best_model.h5
 SavedModel format also saved to: /content/drive/MyDrive/Project-10/best_model_saved
Vectorizer saved to: /content/drive/MyDrive/Project-10/tfidf_vectorizer.pkl


**Validating the Model - Error Check**

In [31]:
from sklearn.metrics import accuracy_score
import numpy as np

# Load the best model
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/best_model.h5')

# Load the extracted features from the CSV file
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')

# Feature Engineering: Convert text to numerical vectors (Example using word embeddings)
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
text_features = vectorizer.fit_transform(df['extracted_text']).toarray()

# Reshape the input data for LSTM
text_features = text_features.reshape(text_features.shape[0], 1, text_features.shape[1])

# Make predictions
y_pred = model.predict(text_features)

# Assuming y_true is the same as text_features for autoencoder-like setup
y_true = text_features.reshape(text_features.shape[0],text_features.shape[2]) # Reshape to match prediction shape

# Calculate accuracy
# Accuracy is not the best metric for regression or autoencoder-like tasks.
# Consider using Mean Squared Error (MSE) or other relevant metrics.
# Convert predictions to class labels if needed.

# Example using MSE (Mean Squared Error):
mse = np.mean(np.square(y_true - y_pred))
print("Mean Squared Error:", mse)

# Example using accuracy (not the best metric for this setup):
# Find the indices of the maximum value
y_pred_class = np.argmax(y_pred, axis=1)
y_true_class = np.argmax(y_true, axis=1)

accuracy = accuracy_score(y_true_class, y_pred_class)
print("Accuracy:", accuracy)


1/1 [==============================] - 0s 230ms/step
Mean Squared Error: 0.002199165720896304
Accuracy: 0.0


In [32]:
import pandas as pd

# Load the extracted features from the CSV file
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')

# Check the data types of all columns
print(df.dtypes)

filename           object
test_name          object
technology         object
value              object
units              object
reference_range    object
extracted_text     object
dtype: object


In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Load and Preprocess Data
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')

# 2. Feature Engineering with TF-IDF (before one-hot encoding)
vectorizer = TfidfVectorizer(max_features=1000)
text_features = vectorizer.fit_transform(df['extracted_text']).toarray()
text_features = text_features.reshape(text_features.shape[0], 1, text_features.shape[1])

# Convert numerical columns to numeric data types
numerical_cols = ['value', 'reference_range']
for col in numerical_cols:
    try:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except ValueError:
        print(f"WARNING: Could not convert column '{col}' to numeric.")

# Handle missing values (if any) - choose one method
# df.dropna(subset=numerical_cols, inplace=True)  # Remove rows with NaN
# df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())  # Impute with mean

# One-hot encode string columns (excluding 'extracted_text')
string_columns_to_encode = [col for col in df.select_dtypes(include=['object']).columns if col != 'extracted_text']
for col in string_columns_to_encode:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    df.drop([col], axis=1, inplace=True)

# 3. Split Data (including text_features)
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    text_features, text_features, test_size=0.2, random_state=42
)

# One-hot encoded data (if needed)
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(
    df, df, test_size=0.2, random_state=42
)

# 4. Build and Train LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_text.shape[1], X_train_text.shape[2])))
model.add(Dense(X_train_text.shape[2]))

model.compile(optimizer='adam', loss='mse')

checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/best_model.h5', monitor='val_loss', save_best_only=True, mode='min'
)

model.fit(
    X_train_text,
    y_train_text,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_text, y_test_text),
    callbacks=[checkpoint],
)

print("Training completed. The best model is saved to /content/drive/MyDrive/best_model.h5")

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.0023 - val_loss: 0.0023
Epoch 2/10
1/1 [==============================] - 0s 52ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 3/10
1/1 [==============================] - 0s 54ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 4/10
1/1 [==============================] - 0s 57ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 5/10
1/1 [==============================] - 0s 71ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 6/10
1/1 [==============================] - 0s 58ms/step - loss: 0.0022 - val_loss: 0.0023
Epoch 7/10
1/1 [==============================] - 0s 59ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 8/10
1/1 [==============================] - 0s 74ms/step - loss: 0.0021 - val_loss: 0.0022
Epoch 9/10
1/1 [==============================] - 0s 67ms/step - loss: 0.0021 - val_loss: 0.0022
Epoch 10/10
1/1 [==============================] - 0s 54ms/step - loss: 0.0021 - val_loss: 0.0022
Training completed. The best mo

**Error and the accuracy of the above model**

In [34]:
# Load the best model
model = load_model('/content/drive/MyDrive/best_model.h5')

# Load the preprocessed data (including one-hot encoded features)
df = pd.read_csv('/content/drive/MyDrive/extracted_features.csv')

# 2. Feature Engineering with TF-IDF (before one-hot encoding)
vectorizer = TfidfVectorizer(max_features=1000)
text_features = vectorizer.fit_transform(df['extracted_text']).toarray()
text_features = text_features.reshape(text_features.shape[0], 1, text_features.shape[1])

# Convert numerical columns to numeric data types
numerical_cols = ['value', 'reference_range']
for col in numerical_cols:
    try:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except ValueError:
        print(f"WARNING: Could not convert column '{col}' to numeric.")

# Handle missing values (if any) - choose one method
# df.dropna(subset=numerical_cols, inplace=True)  # Remove rows with NaN
# df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())  # Impute with mean

# One-hot encode string columns (excluding 'extracted_text')
string_columns_to_encode = [col for col in df.select_dtypes(include=['object']).columns if col != 'extracted_text']
for col in string_columns_to_encode:
    df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    df.drop([col], axis=1, inplace=True)

# 3. Split Data (including text_features)
X_train_text, X_test_text, y_train_text, y_test_text = train_test_split(
    text_features, text_features, test_size=0.2, random_state=42
)
# ... (rest of your data loading and preprocessing code)

# Make predictions
y_pred = model.predict(X_test_text)

# Calculate accuracy (adjust as needed)
mse = np.mean(np.square(y_test_text.reshape(y_test_text.shape[0],y_test_text.shape[2]) - y_pred))
print("Mean Squared Error:", mse)

# You can also try to calculate an accuracy like this:
# Accuracy is not the best metric for regression or autoencoder-like tasks.
# Consider using Mean Squared Error (MSE) or other relevant metrics.
y_pred_class = np.argmax(y_pred, axis=1)
y_true_class = np.argmax(y_test_text.reshape(y_test_text.shape[0], y_test_text.shape[2]), axis=1)

accuracy = accuracy_score(y_true_class, y_pred_class)
print("Accuracy:", accuracy)


1/1 [==============================] - 0s 145ms/step
Mean Squared Error: 0.0022476211628786225
Accuracy: 0.0


**Save the model in pickle file**

In [40]:
import pickle

# Assuming 'model' is your trained LSTM model
# Save the model to a pickle file
with open('/content/drive/MyDrive/trained_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved to /content/drive/MyDrive/trained_model.pkl")


Model saved to /content/drive/MyDrive/trained_model.pkl


**hyper  tune the model to improve accuracy of the model**

In [39]:
!pip install -q -U keras-tuner
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=32),
                   activation='relu',
                   input_shape=(X_train_text.shape[1], X_train_text.shape[2])))
    model.add(Dense(X_train_text.shape[2]))  # Output layer should match input feature size for this autoencoder-like setup

    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop']),
                  loss='mse',  # Use appropriate loss function
                  metrics=['mse']) # Add mse as metric for monitoring

    return model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',  # or 'val_mse' if you use val_mse as metric
    max_trials=5,  # Number of hyperparameter combinations to try
    executions_per_trial=3,  # Number of times to run each trial
    directory='my_dir',
    project_name='helloworld'
)

# Perform hyperparameter search
tuner.search(X_train_text, y_train_text, epochs=10, validation_data=(X_test_text, y_test_text))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best hyperparameters: {best_hps.values}")

# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train_text, y_train_text, epochs=10, validation_data=(X_test_text, y_test_text))


# Evaluate the best model
loss, mse = best_model.evaluate(X_test_text, y_test_text)
print("Loss:", loss)
print("MSE:", mse)

# Save the best model
best_model.save('/content/drive/MyDrive/best_tuned_model.h5')

Trial 5 Complete [00h 00m 12s]
val_loss: 0.0022370093502104282

Best val_loss So Far: 0.0022370093502104282
Total elapsed time: 00h 00m 55s
Best hyperparameters: {'units': 32, 'optimizer': 'adam'}
Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 2/10
1/1 [==============================] - 0s 45ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 3/10
1/1 [==============================] - 0s 49ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 4/10
1/1 [==============================] - 0s 27ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 5/10
1/1 [==============================] - 0s 27ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 6/10
1/1 [==============================] - 0s 40ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 7/10
1/1 [===============

Build a Custom OCR by combining YOLO and Tesseract, to read the specific contents of a Lab

Report and convert it into an editable file. Use  YOLO_V3 to trained on the personal dataset.

Then the coordinates of the detected objects are passed for cropping the detected objects and

storing them in another list. This list is passed through the Tesseract to get the desired output.

**Model**

● You can train a custom YOLO_V3 model using your custom dataset.

● Make a folder named model and put the weights file inside it.

In [ ]:
Dataset_OCR/
├── images/
│   ├── train/
│   │   └── your_training_images.jpg  # (and other images)
│   └── val/
│       └── your_validation_images.jpg  # (and other images)
└── labels/
    ├── train/
    │   └── your_training_labels.txt  # (and other labels)
    └── val/
        └── your_validation_labels.txt  # (and other labels)